In [ ]:
#!pip install --upgrade google-api-python-client
#!pip install --user google-colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###IMPORTANT-> DO NOT RUN THIS PART UNLESS YOU WANT TO CREATE NEW TRAINING TEST AND DEV DATASETS.

### CREATING TEST, TRAIN AND DEV FROM MEDMENTIONS CORPUS

READING FILES: dev, test, train and corpus

In [ ]:
#read file DEV
devTxt="/content/drive/MyDrive/medNer/corpus_pubtator_pmids_dev.txt"
with open(devTxt, "r") as file1:
    devTxtList = file1.readlines()
#delete \n at the end of every line    
for i in range(0,len(devTxtList)):
  devTxtList[i]=devTxtList[i].split('\n')[0]


#read file TEST
testTxt="/content/drive/MyDrive/medNer/corpus_pubtator_pmids_test.txt"
with open(testTxt, "r") as file2:
    testTxtList = file2.readlines()
#delete \n at the end of every line    
for i in range(0,len(testTxtList)):
  testTxtList[i]=testTxtList[i].split('\n')[0]


#read file TRAIN
trngTxt="/content/drive/MyDrive/medNer/corpus_pubtator_pmids_trng.txt"
with open(trngTxt, "r") as file3:
    trngTxtList = file3.readlines()
#delete \n at the end of every line    
for i in range(0,len(trngTxtList)):
  trngTxtList[i]=trngTxtList[i].split('\n')[0]


#read file CORPUS
corpusTxt="/content/drive/MyDrive/medNer/corpus_pubtator.txt"
with open(corpusTxt, "r") as file4:
    corpusTxtList = file4.readlines()


LOADING TOKENIZER

In [ ]:
#DE PRUEBAS
# http://sentiment.christopherpotts.net/code-data/happyfuntokenizing.py -en bertsio motza
import nltk, re, json
import spacy
from collections import Counter
# Ordena garrantzitsua da  (lehenetik azkenera egiten dire parekatzeak)

# Erabiltzaile-izenak batera mantendu (@ ikurraz hasitako edozein token, eta ondoren A-Z, a-z, 0-9)
regexes=(
# zuriunea ez den beste edozer
r"(?:\w+\.\w+)", 
#symbols
r"(?:[\<\>\=\+|\-|\/|\*\\\'\"|\%|\w]+)",   

# hashtag-ak elkarrekin mantendu (# ikurraz hasitako edozein token, eta ondoren A-Z, a-z, 0-9, _, edo -)
r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)",


)

big_regex="|".join(regexes)

my_extensible_tokenizer = re.compile(big_regex, re.VERBOSE | re.I | re.UNICODE)

def my_extensible_tokenize(text):
    return my_extensible_tokenizer.findall(text)

CREATING A FUNCTION TO NORMALIZE ANY STRING USING THE TOKENIZER

In [ ]:
def normalizeString(string):
  #print(string)
  last_char = string[-1]
  isEndOrComma=None

  if last_char == '.':
    isEndOrComma="END"
  if last_char == ',':
    isEndOrComma="COMMA"
    
  string=my_extensible_tokenize(string)
  #print(string)
  if len(string)==0:
    return "", isEndOrComma
  else:
    return string[0], isEndOrComma

In [ ]:
normalizeString("(hola,)")

('hola', None)

CREATING FUNCTIONS FOR CREATING TSV FILES FOR TRAINING DEV AND TEST

In [ ]:
import csv

def textToTokens(title, paper):
  title=title.split()
  paper=paper.split()
  title[0]=title[0].split('|t|')[1]
  paper[0]=paper[0].split('|a|')[1]
  text = title+paper
  #print(text)
  return text
  
def textAndBItoTSV(tsv_writer, text, BI):#

  textCount=0
  for i in BI:
    word = i[0].split()
    CUI=i[1].rstrip("\n")
    #print(word[0])
    #print(text[textCount])
    for w in range(0,len(word)):
      #print("for")
      if textCount<len(text): #puede que sea <=

        #normalize strings
        #print("word[w]"+str(word[w]))
        word[w], isEndOrComma=normalizeString(word[w])
        #print("text[textCount"+str(textCount)+"]"+str(text[textCount]))
        text[textCount], isEndOrComma=normalizeString(text[textCount])

        if word[w]==text[textCount]:
          if w==0:
            #print(word[w])
            #print(text[textCount])
            if word[w]!="" and text[textCount]!="":
              tsv_writer.writerow([text[textCount].lower(), 'B-'+CUI ])
              #print("B")
              #print("word: "+str(word[w]))
              #print("textCount: "+str(text[textCount]))
              if isEndOrComma=="END":
                tsv_writer.writerow([".", 'O'])
                tsv_writer.writerow(["", ''])
              if isEndOrComma=="COMMA":
                tsv_writer.writerow([",", 'O'])

            textCount+=1
            
          else:
            if word[w]!="" and text[textCount]!="":
              tsv_writer.writerow([text[textCount].lower(), 'I-'+CUI])
              #print("I")
              #print("word: "+str(word[w]))
              #print("textCount: "+str(text[textCount]))
              if isEndOrComma=="END":
                tsv_writer.writerow([".", 'O'])
                tsv_writer.writerow(["", ''])
              if isEndOrComma=="COMMA":
                tsv_writer.writerow([",", 'O'])
            textCount+=1
            
        else:
          justEntered=True
          while textCount<len(text)-1:
            #print("while")
            if justEntered==False:
              #print("word[w]"+str(word[w]))
              word[w], isEndOrComma=normalizeString(word[w])
              #print(normalizeString(text[textCount]))
              #print("text[textCount"+str(textCount)+"]"+str(text[textCount]))
              text[textCount], isEndOrComma=normalizeString(text[textCount])
            justEntered=False
            if word[w]!=text[textCount]:
              if word[w]!="" and text[textCount]!="":
                tsv_writer.writerow([text[textCount].lower(), 'O'])
                if isEndOrComma=="END":
                  tsv_writer.writerow([".", 'O'])
                  tsv_writer.writerow(["", ''])
                if isEndOrComma=="COMMA":
                  tsv_writer.writerow([",", 'O'])

              #print("O")
              #print("word: "+str(word[w]))
              #print("textCount: "+str(text[textCount]))
              textCount+=1
            else:
              if w==0:
                #print("B")
                #print("word: "+str(word[w]))
                #print("textCount: "+str(text[textCount]))
                if word[w]!="" and text[textCount]!="":
                  tsv_writer.writerow([text[textCount].lower(), 'B-'+CUI])
                  if isEndOrComma=="END":
                    tsv_writer.writerow([".", 'O'])
                    tsv_writer.writerow(["", ''])
                  if isEndOrComma=="COMMA":
                    tsv_writer.writerow([",", 'O'])
                textCount+=1
                break
              else:
                #print("I")
                #print("word: "+str(word[w]))
                #print("textCount: "+str(text[textCount]))
                if word[w]!="" and text[textCount]!="":
                  tsv_writer.writerow([text[textCount].lower(), 'I-'+CUI])
                  if isEndOrComma=="END":
                    tsv_writer.writerow([".", 'O'])
                    tsv_writer.writerow(["", ''])
                  if isEndOrComma=="COMMA":
                    tsv_writer.writerow([",", 'O'])
                textCount+=1
                break
         
        



CREATING TRAINING SET

In [ ]:
inPaper=False
titleRow=0
paperRow=1
BIbek=[]

#COMMENT NEXT TWO LINES FOR WHOLE CORPUS
#maxAmountOfPapers=100
paperKont=0

with open('/content/drive/MyDrive/medNer/medMentions-train-CUI-version.tsv', 'wt') as out_file:
  tsv_writer = csv.writer(out_file, delimiter='\t')

  for i in range(0,len(corpusTxtList)):
    
    #beginning of paper
    if(inPaper==False):
      #number of paper
      paperNumber=corpusTxtList[i+2].split('\t')[0]
      #title
      currentTitle=corpusTxtList[i]
      #paper
      currentPaper=corpusTxtList[i+1]
      #restarting BIbek
      

    #mid paper
    if titleRow!=i and paperRow!=i and corpusTxtList[i]!='\n':
      #vector for storing begin and inside words
      #print(corpusTxtList[i].split('\t')[-1])
      elemOfBek=[corpusTxtList[i].split('\t')[3],corpusTxtList[i].split('\t')[-1]]
      BIbek.append(elemOfBek)
    inPaper=True
    #end of paper
    if corpusTxtList[i]=='\n':
      titleRow=i+1
      paperRow=i+2
      inPaper=False
      if paperNumber in trngTxtList:
        #COMMENT NEXT LINE FOR WHOLE CORPUS
        #if paperKont<maxAmountOfPapers:
        textAndBItoTSV(tsv_writer, textToTokens(currentTitle, currentPaper),BIbek)
          #COMMENT NEXT LINE FOR WHOLE CORPUS
          #paperKont+=1
      BIbek=[]


CREATING TEST SET

In [ ]:
inPaper=False
titleRow=0
paperRow=1
BIbek=[]

#COMMENT NEXT TWO LINES FOR WHOLE CORPUS
#maxAmountOfPapers=40
paperKont=0

with open('/content/drive/MyDrive/medNer/medMentions-test-CUI-version.tsv', 'wt') as out_file:
  tsv_writer = csv.writer(out_file, delimiter='\t')

  for i in range(0,len(corpusTxtList)):
    
    #beginning of paper
    if(inPaper==False):
      #number of paper
      paperNumber=corpusTxtList[i+2].split('\t')[0]
      #title
      currentTitle=corpusTxtList[i]
      #paper
      currentPaper=corpusTxtList[i+1]
      #restarting BIbek
      

    #mid paper
    if titleRow!=i and paperRow!=i and corpusTxtList[i]!='\n':
      #vector for storing begin and inside words
      elemOfBek=[corpusTxtList[i].split('\t')[3],corpusTxtList[i].split('\t')[-1]]
      BIbek.append(elemOfBek)
    inPaper=True
    #end of paper
    if corpusTxtList[i]=='\n':
      titleRow=i+1
      paperRow=i+2
      inPaper=False
      if paperNumber in testTxtList:
        #COMMENT NEXT LINE FOR WHOLE CORPUS
        #if paperKont<maxAmountOfPapers:
        textAndBItoTSV(tsv_writer, textToTokens(currentTitle, currentPaper),BIbek)
          #COMMENT NEXT LINE FOR WHOLE CORPUS
          #paperKont+=1
      BIbek=[]


CREATING DEV SET

In [ ]:
inPaper=False
titleRow=0
paperRow=1
BIbek=[]

#COMMENT NEXT TWO LINES FOR WHOLE CORPUS
#maxAmountOfPapers=40
paperKont=0

with open('/content/drive/MyDrive/medNer/medMentions-dev-CUI-version.tsv', 'wt') as out_file:
  tsv_writer = csv.writer(out_file, delimiter='\t')

  for i in range(0,len(corpusTxtList)):
    
    #beginning of paper
    if(inPaper==False):
      #number of paper
      paperNumber=corpusTxtList[i+2].split('\t')[0]
      #title
      currentTitle=corpusTxtList[i]
      #paper
      currentPaper=corpusTxtList[i+1]
      #restarting BIbek
      

    #mid paper
    if titleRow!=i and paperRow!=i and corpusTxtList[i]!='\n':
      #vector for storing begin and inside words
      elemOfBek=[corpusTxtList[i].split('\t')[3],corpusTxtList[i].split('\t')[-1]]
      BIbek.append(elemOfBek)
    inPaper=True
    #end of paper
    if corpusTxtList[i]=='\n':
      titleRow=i+1
      paperRow=i+2
      inPaper=False
      if paperNumber in devTxtList:
        #COMMENT NEXT LINE FOR WHOLE CORPUS
        #if paperKont<maxAmountOfPapers:
          textAndBItoTSV(tsv_writer, textToTokens(currentTitle, currentPaper),BIbek)
          #COMMENT NEXT LINE FOR WHOLE CORPUS
          #paperKont+=1

      BIbek=[]


###YOU CAN TEST THE TRAINING PART IF YOU ARE WILLING TO TRAIN AN ALREADY EXISTING MODEL

###TRAINING

Flair 

In [ ]:
pip install --upgrade flair

     |████████████████████████████████| 286kB 5.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 983kB 9.8MB/s 
     |████████████████████████████████| 19.7MB 1.9MB/s 
     |████████████████████████████████| 1.2MB 42.9MB/s 
     |████████████████████████████████| 776.8MB 25kB/s 
     |████████████████████████████████| 798kB 38.2MB/s 
     |████████████████████████████████| 2.1MB 39.9MB/s 
     |████████████████████████████████| 71kB 9.1MB/s 
     |████████████████████████████████| 901kB 40.0MB/s 
     |████████████████████████████████| 3.3MB 30.3MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=25028146c71d41f1dbc750d2d4e2372f620b48f11cfb871e1e7a9ba706e0fae1
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Created wheel for segtok: filename=

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import  WordEmbeddings, StackedEmbeddings#, FlairEmbeddings, TokenEmbeddings
from typing import List

In [ ]:
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '/content/drive/MyDrive/medNer'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='medMentions-train-CUI-version.tsv',
                              test_file='medMentions-test-CUI-version.tsv',
                              dev_file='medMentions-dev-CUI-version.tsv')

# TODO obtain and print corpus statistics (output below obtained with the BASQUE NER corpus)
print(corpus.obtain_statistics())

2021-05-05 14:36:31,007 Reading data from /content/drive/MyDrive/medNer
2021-05-05 14:36:31,014 Train: /content/drive/MyDrive/medNer/medMentions-train-CUI-version.tsv
2021-05-05 14:36:31,018 Dev: /content/drive/MyDrive/medNer/medMentions-dev-CUI-version.tsv
2021-05-05 14:36:31,020 Test: /content/drive/MyDrive/medNer/medMentions-test-CUI-version.tsv
{
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 941,
        "number_of_documents_per_class": {},
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 25187,
            "min": 2,
            "max": 122,
            "avg": 26.766206163655685
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 381,
        "number_of_documents_per_class": {},
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 10302,
            "min": 3,
            "max": 89,
            "avg": 27.03937007874016
        }


In [ ]:
# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)

[b'<unk>', b'O', b'B-C4308010', b'B-C0854135', b'I-C0854135', b'B-C0010674', b'I-C0010674', b'B-C0030705', b'B-C0443252', b'B-C0024115', b'I-C0024115', b'B-C0220921', b'I-C0220921', b'B-C0847557', b'I-C0847557', b'B-C3160731', b'I-C3160731', b'B-C1521828', b'B-C4086268', b'B-C3640077', b'I-C3640077', b'B-C1707689', b'I-C1707689', b'B-C0597298', b'I-C4308010', b'B-C0035204', b'I-C0035204', b'B-C2603343', b'B-C1292732', b'B-C3845273', b'B-C0021149', b'B-C0001779', b'B-C0599755', b'B-C0001675', b'B-C0475309', b'B-C0032520', b'I-C0032520', b'B-C3899368', b'I-C3899368', b'B-C0444245', b'I-C0444245', b'B-C0019994', b'I-C0019994', b'B-C1413365', b'B-C0243067', b'B-C0876973', b'I-C0876973', b'B-C0033809', b'B-C0086582', b'B-C0026882', b'I-C0026882', b'B-C0699748', b'B-C0042382', b'B-C0037114', b'B-C1706182', b'B-C2349975', b'B-C1705822', b'B-C2828363', b'I-C2828363', b'B-C0085104', b'I-C0085104', b'B-C0183210', b'I-C0183210', b'B-C0444498', b'I-C0444498', b'B-C0185125', b'B-C0439801', b'B-C001

You can choose the embeddings. The ideal is the following combination:FlairEmbeddings('pubmed-forward'), FlairEmbeddings('pubmed-backward'), WordEmbeddings("pubmed"). But due to memory issues just WordEmbeddings("pubmed") is the best choice.

In [ ]:
embeddings : StackedEmbeddings = StackedEmbeddings([
                                        WordEmbeddings('glove'),
                                        #FlairEmbeddings('news-forward'),
                                        #FlairEmbeddings('news-backward')
                                       ])

2021-05-05 14:36:35,255 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmplmshuu0k


100%|██████████| 160000128/160000128 [00:17<00:00, 9353451.79B/s]

2021-05-05 14:36:53,201 copying /tmp/tmplmshuu0k to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2021-05-05 14:36:53,522 removing temp file /tmp/tmplmshuu0k
2021-05-05 14:36:55,680 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmpj5dyl74c


100%|██████████| 21494764/21494764 [00:04<00:00, 4994067.85B/s]

2021-05-05 14:37:00,815 copying /tmp/tmpj5dyl74c to cache at /root/.flair/embeddings/glove.gensim
2021-05-05 14:37:00,847 removing temp file /tmp/tmpj5dyl74c


In [ ]:
#from gensim.models import KeyedVectors
#word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
#x = word2vec.word_vec("test")
#EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'

In [ ]:
from flair.models import SequenceTagger

tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)
print(tagger)


SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 64, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=128, out_features=5937, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)


In [ ]:
from flair.trainers import ModelTrainer
trainer : ModelTrainer = ModelTrainer(tagger, corpus)
trainer.train('/content/drive/MyDrive/medNer',
              learning_rate=0.1,
              mini_batch_size=16,
              max_epochs=10)

2021-05-05 14:37:05,499 ----------------------------------------------------------------------------------------------------
2021-05-05 14:37:05,502 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 64, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=128, out_features=5937, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2021-05-05 14:37:05,505 ----------------------------------------------------------------------------------------------------
2021-05-05 14:37:05,509 Corpus: "Corpus: 941 train + 380 dev + 381 test sentences"
2021-05-05 14:37:05,512 ----------------------------------------------------------------------------------------------------
2021-05-05 14:37:05,517 Parameters:
2021-05-05 14:37:05,519  - learning_

RuntimeError: ignored

LOAD MODEL

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
# load the trained model
model = SequenceTagger.load('/content/drive/MyDrive/UNI INFOR/Cuarto/HP/9.Flair/final-model.pt')


RESULTS

In [ ]:
from segtok.segmenter import split_single
#read file
Text="/content/drive/MyDrive/UNI INFOR/Cuarto/HP/9.Flair/eusko-jaurlaritza.txt"
with open(Text, "r") as file1:
    FileasList = file1.readlines()
TextStr= FileasList[0]

#NER
#2.Whole text predict
TextStrAll=Sentence(TextStr)
model.predict(TextStrAll)
print(TextStrAll.to_tagged_string())